In [1]:
import urllib3
import os

# data = urllib3.response("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data")
os.popen('wget -c https://raw.githubusercontent.com/amueller/advanced_training/master/data/adult.csv')

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("adult.data.csv")
data.head(50)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


<hr>
<h2>1. How many men and women (sex feature) are represented in this dataset?</h2>

In [4]:
data['sex'].value_counts()

Male      21790
Female    10771
Name: sex, dtype: int64

<hr>
<h1>2. What is the average age (age feature) of women?</h1>

In [5]:
data.loc[data['sex'] == 'Female', 'age'].shape

(10771,)

In [6]:
data.loc[data['sex'] == 'Female', 'age'].mean()


36.85823043357163

<hr>
<h1>3. What is the proportion of German citizens (native-country feature)?</h1>

In [7]:
data[data['native-country'] == 'Germany'].shape[0]/ data.shape[0]

0.004207487485028101

<hr>
<h1>4-5. What are mean value and standard deviation of the age of those who recieve more than 50K per year (salary feature) and those who receive less than 50K per year?
</h1>

In [8]:
gt = data.loc[data['salary'] == '>50K', 'age']
lt = data.loc[data['salary'] == '<=50K', 'age']
print(gt.mean(), gt.std())
print(lt.mean(), lt.std())

44.24984058155847 10.51902771985177
36.78373786407767 14.020088490824813


<hr>
<h1>
6. Is it true that people who receive more than 50k have at least high school education? (education - Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters or Doctorate feature)
</h1>

In [9]:
data.loc[data['salary'] == '>50K', 'education'].unique()

array(['HS-grad', 'Masters', 'Bachelors', 'Some-college', 'Assoc-voc',
       'Doctorate', 'Prof-school', 'Assoc-acdm', '7th-8th', '12th',
       '10th', '11th', '9th', '5th-6th', '1st-4th'], dtype=object)

<hr>
<h1>
7. Display statistics of age for each race (race feature) and each gender. Use groupby() and describe(). Find the maximum age of men of Amer-Indian-Eskimo race.
</h1>

In [10]:
for (race, sex), sub_df in data.groupby(['race', 'sex']):
    print("Race: {0}, sex: {1}".format(race, sex))
    print(sub_df.shape)

Race: Amer-Indian-Eskimo, sex: Female
(119, 15)
Race: Amer-Indian-Eskimo, sex: Male
(192, 15)
Race: Asian-Pac-Islander, sex: Female
(346, 15)
Race: Asian-Pac-Islander, sex: Male
(693, 15)
Race: Black, sex: Female
(1555, 15)
Race: Black, sex: Male
(1569, 15)
Race: Other, sex: Female
(109, 15)
Race: Other, sex: Male
(162, 15)
Race: White, sex: Female
(8642, 15)
Race: White, sex: Male
(19174, 15)


In [11]:
data.groupby(['race', 'sex']).max()

age    workclass   fnlwgt     education  \
race               sex                                               
Amer-Indian-Eskimo Female   80    State-gov   445168  Some-college   
                   Male     82    State-gov   356015  Some-college   
Asian-Pac-Islander Female   75    State-gov   379046  Some-college   
                   Male     90  Without-pay   506329  Some-college   
Black              Female   90    State-gov   930948  Some-college   
                   Male     90  Without-pay  1268339  Some-college   
Other              Female   74    State-gov   388741  Some-college   
                   Male     77    State-gov   481175  Some-college   
White              Female   90  Without-pay  1484705  Some-college   
                   Male     90  Without-pay  1455435  Some-college   

                           education-num marital-status        occupation  \
race               sex                                                      
Amer-Indian-Eskimo Female             16        Widowed  Transport-moving   
                   Male               16        Widowed  Transport-moving   
Asian-Pac-Islander Female             15        Widowed  Transport-moving   
                   Male               16        Widowed  Transport-moving   
Black              Female             16        Widowed  Transport-moving   
                   Male               16        Widowed  Transport-moving   
Other              Female             14        Widowed      Tech-support   
                   Male               16        Widowed  Transport-moving   
White              Female             16        Widowed  Transport-moving   
                   Male               16        Widowed  Transport-moving   

                          relationship  capital-gain  capital-loss  \
race               sex                                               
Amer-Indian-Eskimo Female         Wife         15024          1721   
                   Male      Unmarried         27828          1980   
Asian-Pac-Islander Female         Wife         99999          2258   
                   Male      Unmarried         99999          2457   
Black              Female         Wife         99999          4356   
                   Male      Unmarried         99999          2824   
Other              Female         Wife          7688          1740   
                   Male      Unmarried         99999          2179   
White              Female         Wife         99999          4356   
                   Male           Wife         99999          3770   

                           hours-per-week native-country salary  
race               sex                                           
Amer-Indian-Eskimo Female              84  United-States   >50K  
                   Male                84  United-States   >50K  
Asian-Pac-Islander Female              99        Vietnam   >50K  
                   Male                99        Vietnam   >50K  
Black              Female              99  United-States   >50K  
                   Male                99  United-States   >50K  
Other              Female              65  United-States   >50K  
                   Male                98  United-States   >50K  
White              Female              99     Yugoslavia   >50K  
                   Male                99     Yugoslavia   >50K

<hr>
<h1>8. Among whom the proportion of those who earn a lot(>50K) is more: among married or single men (marital-status feature)? Consider married those who have a marital-status starting with Married (Married-civ-spouse, Married-spouse-absent or Married-AF-spouse), the rest are considered bachelors.</h1>

In [12]:
data.loc[(data['sex'] == 'Male') &
     (data['marital-status'].isin(['Never-married', 
                                   'Separated', 
                                   'Divorced',
                                   'Widowed'])), 'salary'].value_counts()

<=50K    7552
>50K      697
Name: salary, dtype: int64

In [13]:
data.loc[(data['sex'] == 'Male') &
     (data['marital-status'].str.startswith('Married')), 'salary'].value_counts()

<=50K    7576
>50K     5965
Name: salary, dtype: int64

In [14]:
data['marital-status'].value_counts()

Married-civ-spouse       14976
Never-married            10683
Divorced                  4443
Separated                 1025
Widowed                    993
Married-spouse-absent      418
Married-AF-spouse           23
Name: marital-status, dtype: int64

<hr><h1>9. What is the maximum number of hours a person works per week (hours-per-week feature)? How many people work such a number of hours and what is the percentage of those who earn a lot among them?</h1>

In [15]:
max_hours = data['hours-per-week'].max()
count_max_hours = data[data['hours-per-week'] == max_hours].shape[0]
rich_count = data[(data['hours-per-week'] == max_hours) & (data['salary'] == '>50K')].shape[0]
percent = 100 * rich_count / count_max_hours
print(max_hours, count_max_hours, rich_count, percent, sep='\n')

99
85
25
29.41176470588235


<hr>
<h1>10. Count the average time of work (hours-per-week) those who earning a little and a lot (salary) for each country (native-country).</h1>

In [16]:
pd.crosstab(data['native-country'], data['salary'], 
           values=data['hours-per-week'], aggfunc=np.mean).T

native-country,?,Cambodia,Canada,China,Columbia,Cuba,Dominican-Republic,Ecuador,El-Salvador,England,...,Portugal,Puerto-Rico,Scotland,South,Taiwan,Thailand,Trinadad&Tobago,United-States,Vietnam,Yugoslavia
salary,,,,,,,,,,,,,,,,,,,,,
<=50K,40.164760,41.416667,37.914634,37.381818,38.684211,37.985714,42.338235,38.041667,36.030928,40.483333,...,41.939394,38.470588,39.444444,40.15625,33.774194,42.866667,37.058824,38.799127,37.193548,41.6
>50K,45.547945,40.000000,45.641026,38.900000,50.000000,42.440000,47.000000,48.750000,45.000000,44.533333,...,41.500000,39.416667,46.666667,51.43750,46.800000,58.333333,40.000000,45.505369,39.200000,49.5


<hr>

<h1>GROUP BY</h1>

In [17]:
data.groupby('race').agg({'age': ['min', 'max', 'mean']})

age               
                   min max       mean
race                                 
Amer-Indian-Eskimo  17  82  37.173633
Asian-Pac-Islander  17  90  37.746872
Black               17  90  37.767926
Other               17  77  33.457565
White               17  90  38.769881

In [18]:
data.groupby('workclass').agg({'age': ['min', 'max', 'mean']})

age               
                 min max       mean
workclass                          
?                 17  90  40.960240
Federal-gov       17  90  42.590625
Local-gov         17  90  41.751075
Never-worked      17  30  20.571429
Private           17  90  36.797585
Self-emp-inc      17  84  46.017025
Self-emp-not-inc  17  90  44.969697
State-gov         17  81  39.436055
Without-pay       19  72  47.785714

<hr>
<h1>JOIN</h1>

In [19]:
os.popen('wget -c https://github.com/shanealynn/Pandas-Merge-Tutorial/blob/master/user_usage.csv')
os.popen('wget -c https://github.com/shanealynn/Pandas-Merge-Tutorial/blob/master/user_device.csv')
os.popen('wget -c https://github.com/shanealynn/Pandas-Merge-Tutorial/blob/master/android_devices.csv')

In [29]:
user_usage = pd.read_csv('user_usage.csv')
user_device = pd.read_csv('user_device.csv')
android_devices = pd.read_csv('android_devices.csv')

In [30]:
print(user_usage.head())
print(user_device.head())
print(android_devices.head())

   outgoing_mins_per_month  outgoing_sms_per_month  monthly_mb  use_id
0                    21.97                    4.82     1557.33   22787
1                  1710.08                  136.88     7267.55   22788
2                  1710.08                  136.88     7267.55   22789
3                    94.46                   35.17      519.12   22790
4                    71.59                   79.26     1557.33   22792
   use_id  user_id platform  platform_version     device  use_type_id
0   22782    26980      ios              10.2  iPhone7,2            2
1   22783    29628  android               6.0    Nexus 5            3
2   22784    28473  android               5.1   SM-G903F            1
3   22785    15200      ios              10.2  iPhone7,2            3
4   22786    28239  android               6.0  ONE E1003            1
  Retail Branding Marketing Name    Device                      Model
0             NaN            NaN    AD681H  Smartfren Andromax AD681H
1             

In [62]:
import time

begin_pd = time.time()
result = pd.merge(user_usage,
                 user_device[['use_id', 'platform', 'device']],
                 on='use_id')
finish_pd = (time.time() - begin_pd)*1_000
print(finish_pd, 'ms')
result.head()

5.616903305053711 ms


,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


In [63]:
import pandasql as ps

begin_ps = time.time()
aggr_query = """
    SELECT user_usage.outgoing_mins_per_month,
           user_usage.outgoing_sms_per_month,
           user_usage.monthly_mb,
           user_usage.use_id,
           user_device.platform,
           user_device.device
    FROM user_usage JOIN user_device on user_usage.use_id=user_device.use_id
"""
finish_ps = (time.time() - begin_ps)* 1_000
print(finish_ps, 'ms')
ps.sqldf(aggr_query, locals())

0.15687942504882812 ms


,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,platform,device
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F
5,71.59,79.26,1557.33,22793,android,SM-G361F
6,71.59,79.26,519.12,22794,android,SM-G361F
7,71.59,79.26,519.12,22795,android,SM-G361F
8,30.92,22.77,3114.67,22799,android,ONEPLUS A3003
9,69.80,14.70,25955.55,22801,android,GT-I9505


In [64]:
print('pandasql быстрее pandas в', finish_pd/finish_ps, 'раз')

time pd / time psql 35.80395136778115
